In [15]:
import numpy as np
import open3d as o3d
import pathlib
import sys
import math
import itertools
import matplotlib.pyplot as plt
import copy

from collections import Counter

In [2]:
print(f"Numpy version: {np.__version__}")
print(f"Open3D version: {o3d.__version__}")
print(f"Python version: {sys.version[:6]}") 

Numpy version: 1.22.3
Open3D version: 0.17.0
Python version: 3.8.16


## Loading in Point cloud data

Problems encountered: Trouble cleaning with scans with a depth perception problem, might need to investigate order of code execution

Todo: Remove more of the floor manually before applying DBSCAN

Trouble with: 
P003, 
P010 (ground isnt cleaned??) 
P011 (weird capture?) 
P013 (ground isnt cleaned??)
P014 (ground isnt cleaned??)
P015 (ground)
P017


P005 / P006 have some spurious points around its edges, especially around the feet.

In [44]:
# Load ply file
path = "./data/Alex_crop.ply"

def load(path: str) -> o3d.cpu.pybind.geometry.PointCloud:
    '''
        Loads a ply file provided a valid path with robust error checking
        
        Checks file path exists and is a ply file that is not empty
        
        Args:
            path: string path to a .ply file
        
        Returns:
            pcd: non-empty point cloud data object
    '''
    
    # checks if file exists
    if not pathlib.Path(path).exists(): 
        raise FileNotFoundError("File not found.")
    
    # checking correct file format
    if pathlib.Path(path).suffix != '.ply':
        raise ValueError(f"Expected a .ply file, got {pathlib.Path(path).suffix}")
    
    # check if file is not empty
    if pathlib.Path(path).stat().st_size == 0:
        raise ValueError("File is empty.")
    
    pcd = o3d.io.read_point_cloud(path)
    
    # checks pt cloud is not empty
    if not np.asarray(pcd.points).size:
        raise ValueError("Point cloud has no points.")
    
    # pts and colors must match (not too important, but mismatches may lead to incorrect results when vis or analysing)
    if np.asarray(pcd.points).shape[0] != np.asarray(pcd.colors).shape[0]:
        raise ValueError("Point cloud has mismatch between points and colors.")
    
    return pcd # should pass all error cases

pcd = load(path)

print("PLY file loaded.")
print('Shape of points', np.asarray(pcd.points).shape)
print('Shape of colors', np.asarray(pcd.colors).shape)

PLY file loaded.
Shape of points (5348000, 3)
Shape of colors (5348000, 3)


In [45]:
pcd_tmp = copy.deepcopy(pcd)

In [6]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [5]:
pcd_vis = copy.deepcopy(pcd) # need to do this to create a deep copy of pcd

def viewClustersViaColours(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
        Visualising pcd coloured by cluster applying DBSCAN (Density-Based Spatial Clustering of Applications with Noise) algorithm
        Alter cluster_dbscan() params for different results
            eps: max dist between two samples for them to be considered same neighbourhood
            min_points: min number of points required to form a cluster
    '''
    
    labels = np.array(pcd.cluster_dbscan(eps=0.03, min_points=3, print_progress=True))
    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1)) # avoids div error
    colors[labels < 0] = 0 # label=-1 indicates noise
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcd])

viewClustersViaColours(pcd_vis)

point cloud has 21 clusters


In [54]:
from ipywidgets import interact, interactive, Button, FloatSlider, FloatText
from ipywidgets import jslink as link
from IPython.display import display

# holds final state of point cloud
final_pcd = None

def create_grid_on_plane(thresh, axis, color):
    lines = []
    colors = []
    for i in np.linspace(-1, 1, 10):  # Adjust these values as per your requirements
        # vertical lines
        start = [i, thresh, -1] if axis == 'y' else ([thresh, i, -1] if axis == 'x' else [-1, i, thresh])
        end = [i, thresh, 1] if axis == 'y' else ([thresh, i, 1] if axis == 'x' else [1, i, thresh])
        lines.append([start, end])
        colors.append(color)
        # horizontal lines
        start = [-1, thresh, i] if axis == 'y' else ([thresh, -1, i] if axis == 'x' else [i, -1, thresh])
        end = [1, thresh, i] if axis == 'y' else ([thresh, 1, i] if axis == 'x' else [i, 1, thresh])
        lines.append([start, end])
        colors.append(color)

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.array(lines).reshape(-1, 3)),
        lines=o3d.utility.Vector2iVector(np.array([[i, i+1] for i in range(0, len(lines)*2, 2)])),
    )

    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

pcd_tmp = np.asarray(pcd.points)

min_x, min_y, min_z = pcd_tmp.min(axis=0)
max_x, max_y, max_z = pcd_tmp.max(axis=0)

def view_and_adjust_threshold(thresh_x=min_x, thresh_x_dup=max_x, thresh_y=min_y, thresh_y_dup=max_y, thresh_z=min_z):
    global final_pcd 

    pcd_tmp = np.asarray(pcd.points)
    # pcd_tmpc = np.asarray(pcd.colors)

    indices = (((pcd_tmp[:,0] > thresh_x) & (pcd_tmp[:,0] < thresh_x_dup)) & 
               ((pcd_tmp[:,1] > thresh_y) & (pcd_tmp[:,1] < thresh_y_dup)) & 
               (pcd_tmp[:,2] > thresh_z))
    pcd_tmp_tmp = pcd_tmp[indices]
    # pcd_tmp_tmpc = pcd_tmpc[indices]

    filtered = o3d.geometry.PointCloud()
    filtered.points = o3d.utility.Vector3dVector(pcd_tmp_tmp)
    # filtered.colors = o3d.utility.Vector3dVector(pcd_tmp_tmpc)
    
    final_pcd = filtered
    
    plane_x = create_grid_on_plane(thresh_x, 'x', [1, 0, 0])  # red color for x plane
    plane_y = create_grid_on_plane(thresh_y, 'y', [0, 1, 0])  # green color for y plane
    plane_z = create_grid_on_plane(thresh_z, 'z', [0, 0, 1])  # blue color for z plane
    
    plane_x_dup = create_grid_on_plane(thresh_x_dup, 'x', [1, 0, 0])  # duplicate x plane
    plane_y_dup = create_grid_on_plane(thresh_y_dup, 'y', [0, 1, 0])  # duplicate y plane
    
    ## visualise origin pt
    origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0,0,0]) 
    o3d.visualization.draw_geometries([final_pcd, origin, plane_x, plane_y, plane_z, plane_x_dup, plane_y_dup])

    # o3d.visualization.draw_geometries([final_pcd, plane_x, plane_y, plane_z, plane_x_dup, plane_y_dup])

def save_pcd(b):
    if final_pcd is not None:
        o3d.io.write_point_cloud("final.ply", final_pcd)
        print("Point cloud saved!")
    else:
        print("No point cloud to save.")

save_button = Button(description="Save point cloud")
save_button.on_click(save_pcd)  

pcd_tmp = np.asarray(pcd.points)

min_x, min_y, min_z = pcd_tmp.min(axis=0)
max_x, max_y, max_z = pcd_tmp.max(axis=0)

slider_x = FloatSlider(min=min_x, max=max_x, step=0.001, value=min_x, description="x")
text_x = FloatText(value=min_x, description="x")
link((slider_x, 'value'), (text_x, 'value'))

slider_x_dup = FloatSlider(min=min_x, max=max_x, step=0.001, value=max_x, description="x_dup")
text_x_dup = FloatText(value=max_x, description="x_dup")
link((slider_x_dup, 'value'), (text_x_dup, 'value'))

slider_y = FloatSlider(min=min_y, max=max_y, step=0.001, value=min_y, description="y")
text_y = FloatText(value=min_y, description="y")
link((slider_y, 'value'), (text_y, 'value'))

slider_y_dup = FloatSlider(min=min_y, max=max_y, step=0.001, value=max_y, description="y_dup")
text_y_dup = FloatText(value=max_y, description="y_dup")
link((slider_y_dup, 'value'), (text_y_dup, 'value'))

slider_z= FloatSlider(min=min_z, max=max_z, step=0.001, value=min_z, description="z")
text_z = FloatText(value=round(min_z, 2), description="z")
link((slider_z, 'value'), (text_z, 'value'))

interactive(view_and_adjust_threshold, 
         thresh_x=slider_x, 
         thresh_x_dup=slider_x_dup, 
         thresh_y=slider_y, 
         thresh_y_dup=slider_y_dup, 
         thresh_z=slider_z)

# display(text_x, text_x_dup, text_y, text_y_dup, text_z, interact, save_button)


interactive(children=(FloatSlider(value=-0.8107507814104118, description='x', max=0.4033583868575452, min=-0.8…

In [53]:
from sklearn.decomposition import PCA
import numpy as np
from scipy.spatial.transform import Rotation

valid_points = np.asarray(pcd.points)

# 1. Center the data
mean_cleaned = np.mean(valid_points, axis=0)
centered_data_cleaned = valid_points - mean_cleaned

# 2. Scale the data
scaling_factor = np.max(np.abs(centered_data_cleaned))
scaled_data = centered_data_cleaned / scaling_factor

# 3. Compute the covariance matrix
covariance_matrix_scaled = np.cov(scaled_data, rowvar=False)

# 4. Compute the singular value decomposition
_, _, Vt_scaled = np.linalg.svd(covariance_matrix_scaled)

# 5. The principal components are the rows of Vt
principal_components_scaled = Vt_scaled

def compute_rotation_matrix_to_z_axis(normal_vector):
    """Compute the rotation matrix that aligns the given vector with the z-axis."""
    
    # Vector we want to align the normal to (z-axis)
    target_vector = np.array([1, 0, 0])
    
    # Compute the rotation vector
    rotation_vector = np.cross(normal_vector, target_vector)
    rotation_vector_magnitude = np.linalg.norm(rotation_vector)
    
    # Compute the rotation angle
    dot_product = np.dot(normal_vector, target_vector)
    cos_angle = dot_product / (np.linalg.norm(normal_vector) * np.linalg.norm(target_vector))
    rotation_angle = np.arccos(cos_angle)
    
    # Get the rotation matrix
    rotation = Rotation.from_rotvec(rotation_angle * rotation_vector / rotation_vector_magnitude)
    rotation_matrix = rotation.as_matrix()
    
    return rotation_matrix

# Get the normal vector of the primary plane of variation (third principal component)
normal_vector = principal_components_scaled[2, :]

# Compute the rotation matrix
rotation_matrix_to_z = compute_rotation_matrix_to_z_axis(normal_vector)

# Rotate the point cloud to align with the z-axis
aligned_points = valid_points @ rotation_matrix_to_z.T

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0,0,0]) 

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(aligned_points)
pcd.colors = o3d.utility.Vector3dVector(np.asarray(pcd.colors))
o3d.visualization.draw_geometries([origin, pcd])

In [47]:
interactive(view_and_adjust_threshold, 
         thresh_x=slider_x, 
         thresh_x_dup=slider_x_dup, 
         thresh_y=slider_y, 
         thresh_y_dup=slider_y_dup, 
         thresh_z=slider_z)

display(text_x, text_x_dup, text_y, text_y_dup, text_z, interact, save_button)

FloatText(value=-1.123749355465542, description='x')

FloatText(value=0.805807093087455, description='x_dup')

FloatText(value=-1.1921722000838315, description='y')

FloatText(value=1.1518723234370207, description='y_dup')

FloatText(value=-0.40335838685753456, description='z')

Button(description='Save point cloud', style=ButtonStyle())

In [27]:
from ipywidgets import interactive, Button

def my_function(x):
    print(f"Using value: {x}")

slider_x = FloatSlider(min=0, max=10, step=0.1, value=5, description="X Value:")
w = interactive(my_function, x=slider_x)

def on_button_click(b):
    view_and_adjust_threshold()

button = Button(description="Visualize")
button.on_click(on_button_click)

display(slider_x, button)


FloatSlider(value=5.0, description='X Value:', max=10.0)

Button(description='Visualize', style=ButtonStyle())

In [28]:
o3d.visualization.draw_geometries([final_pcd])

In [14]:
# note: pcd = final_pcd creates a shallow copy, only stores reference to cropped pcd so any cahnges to pcd = changes to final_pcd   
# Use deepcopy() to create an independent / deep copy

pcd = copy.deepcopy(final_pcd) 

In [30]:
o3d.visualization.draw_geometries([pcd])

# Preprocessing

In [31]:
# Perform plane segmentation
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                        ransac_n=3,
                                        num_iterations=2000)
# Here, distance_threshold is the maximum distance a point can be from the plane to be considered an inlier,
# ransac_n is the number of initial points to sample,
# num_iterations is the number of iterations the algorithm performs.

# inliers contains the indices of the points that belong to the plane
# We can use this to create a new point cloud containing only these points:
plane = pcd.select_by_index(inliers)
plane.paint_uniform_color([0,1,0])

# We can also create a point cloud containing everything else:
pcd = pcd.select_by_index(inliers, invert=True)

# Visualize
o3d.visualization.draw_geometries([pcd])

In [32]:
def downsample_clean(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Performs downsampling, outlier removal and bounding box removal
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: None 
    '''

    # voxel downsampling: reducing overall num of pts
    voxel_size = 0.01
    downsampled = pcd.voxel_down_sample(voxel_size) 
    
    # outlier cleaning
    _, ind = downsampled.remove_statistical_outlier(nb_neighbors=100, std_ratio=2.0)
    cleaned_pcd = downsampled.select_by_index(ind)
    
    # paint removal parts
    epsilon = 0.01
    points = np.asarray(pcd.points)
    indices = np.where(np.abs(points[:, 2] < epsilon))[0]
    
    pcd_in_color = pcd.select_by_index(indices)
    pcd_in_color.paint_uniform_color([1,1,0])
    pcd = pcd.select_by_index(indices, invert=True)
    
    o3d.visualization.draw_geometries([pcd])

    return pcd

downsample_clean(pcd)

PointCloud with 11405 points.

In [33]:
pcd = downsample_clean(pcd)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [34]:

def isolateLargestCluster(pcd: o3d.cpu.pybind.geometry.PointCloud, labels: np.ndarray) -> None:
    '''
    Uses DBSCAN to group and isolate the largest point cloud 
       
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
        labels (np.ndarray): Array of clusters identified by labels
    
    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud
    '''

    # finding the label of largest cluster + ignoring noisy points labeled -1
    counts = Counter(labels)
    largest_cluster_label = max(counts.items(), key=lambda x: x[1] if x[0] != -1 else -1)[0]

    # get all pts / colors of largest cluster
    largest_cluster_points = np.array(pcd.points)[labels == largest_cluster_label]
    largest_cluster_colors = np.array(pcd.colors)[labels == largest_cluster_label]
    
    # new point cloud from the largest cluster pts w/ colors
    largest_cluster_pcd = o3d.geometry.PointCloud()
    largest_cluster_pcd.points = o3d.utility.Vector3dVector(largest_cluster_points)
    largest_cluster_pcd.colors = o3d.utility.Vector3dVector(largest_cluster_colors)
    
    return largest_cluster_pcd
    
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=3, print_progress=True))
pcd = isolateLargestCluster(pcd, labels)
o3d.visualization.draw_geometries([pcd])